In [3]:
from IPython.core.display import HTML
HTML("<style>" + open("style.css").read() + "</style>")

<div class="headline">
Language Technology / Sprachtechnologie
<br><br>
Wintersemester 2020/2021
</div>
<br>
<div class="description">
    Übung zum Thema <i id="topic">"Sentiment Analysis"</i>
    <br><br>
    Deadline Abgabe: <i #id="submission">12.02.2021</i>
</div>

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from os.path import join
from nltk.corpus import stopwords

In [ ]:
df = pd.read_csv(join('data', 'amazon_reviews_sentiments.tsv'), names=['Statement', 'label'], sep='\t')

statements = df['Statement']
labels = df['label']

train_statements = statements[:-200]
train_labels = labels[:-200]
test_statements = statements[-200:]
test_labels = labels[-200:]
df.head()

# Präsenzübung

### Try with tf-IDF 

<div class="task_description">
    <i class="task">Task 11.5:</i> <br>
</div>

Train the naive bayes classifier using the TF_IDF feature set of the train data. <br>
Then test the classifier's performance on the test set.  

<div class="task_description">
   <i class="subtask">11.5.1</i> <i class="l1">L1</i> <br>
</div>

Explain the following code.  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), lowercase=True)  
features_train = vectorizer.fit_transform(train_statements).toarray()
features_test = vectorizer.transform(test_statements).toarray() 
print(features_train)

X_train = features_train
y_train = train_labels
X_test = features_test
y_test = test_labels
X_train.shape

<strong style="color: blue">Lösung:</strong>

Create tf-idf features, remove nltk english stopwords,  lowercasing for train and test statements.

<div class="task_description">
   <i class="subtask">11.5.2</i> <i class="l2">L2</i> <br>
</div>

Apply the naive bayes algorithm and train the classifier using the train set using the 'fit' method. 

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
# TODO train the classifier on the train set.


<strong style="color: blue">Lösung:</strong>

In [ ]:
from sklearn.naive_bayes import GaussianNB

# fast better classifier
clf = GaussianNB()
clf.fit(X_train, y_train)


### Try naive bayes algorithm

<div class="task_description">
   <i class="subtask">11.5.3</i> <i class="l2">L2</i> <br>
</div>

Evaluate the classifier on the test set using confusion metric, accuracy and classification report.

<strong style="color: blue">Lösung:</strong>

In [ ]:
predictions = clf.predict(X_test)

cf = confusion_matrix(y_test,predictions)
print(cf)

In [ ]:
import seaborn as sns
sns.heatmap(cf, cmap="GnBu", annot=True, fmt='g');

In [ ]:
accuracy_score(y_test, predictions) 

In [ ]:
print(classification_report(y_test,predictions))

<div class="task_description">
   <i class="subtask">11.5.4</i> <i class="l2">L2</i> <br>
</div>

Find three statements that the classifier fails. Please run the following code to test the classifier. Give a justification for the classifier failure. 

In [ ]:
# Process user input. Processing continues until the user says goodbye. 
text = ""
while text != "goodbye":
    # Read user input
    text = input()
    test_features = vectorizer.transform([text]).toarray()  
    print('Positive Sentiment' if clf.predict(test_features) == 1 else 'Negative sentiment')

<strong style="color: blue">Lösung:</strong>

i am sad –– *sad* has a low tf-idf score for negative sentiment text

product is shit –– *shit* is unknown to classifier

i am not satisfied with the product –– *not* is is not considered as removed in stopwords

<div class="task_description">
   <i class="subtask">11.5.5</i> <i class="l2">L2</i> <br>
</div>

The following code snippet generates false positives and false negatives. Please observe the classifier's first ten wrong judgements and describe them briefly.

In [ ]:
# checkout false positive examples

no_example = 10


for i, sample in enumerate(X_test):
    if y_test.values[i] != int(clf.predict([sample])[0]):
        print("Text: %s" %test_statements.values[i])
        print("Ground Truth: %s" %y_test.values[i])
        print("Prediction: %s\n" %clf.predict([sample])[0])
        no_example -= 1
        if no_example == 0:
            break
    

<strong style="color: blue">Lösung:</strong>

The classifier doesn't work very well with negation in the statements


## Try with word embeddings -word2vec

<div class="task_description">
    <i class="task">Task 11.6:</i> <br>
</div>

<div class="task_description">
   <i class="subtask">11.6.1</i> <i class="l2">L2</i> <br>
</div>

Downloaded the pretrained google news word2vec word embeddings from <br><br> https://drive.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM <br><br> Run the folloing code and explain it.

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) #limit

<strong style="color: blue">Lösung:</strong>

The gensim module is loading the google word2vec model

<div class="task_description">
   <i class="subtask">11.6.2</i> <i class="l3">L3</i> <br>
</div>

Generate the embeddding features for test statements. Please remove stopwords (see the train statements example).

In [ ]:
embedding_features_train = []
for sent in train_statements:
    sent = [x.lower() for x in sent.split(' ') if x not in stopwords.words('english')]
    for i in range(len(sent)):
        sent[i] = model[sent[i]] if sent[i] in model.vocab else np.zeros(300)
    embedding_features_train.append(np.mean(sent, axis=0))

embedding_features_train = np.array(embedding_features_train)

In [ ]:
embedding_features_test = []
for sent in test_statements:
    #TODO

embedding_features_test = np.array(embedding_features_test)

In [ ]:
X_train_emb = embedding_features_train
y_train_emb = train_labels

X_test_emb = embedding_features_test
y_test_emb = test_labels

<strong style="color: blue">Lösung:</strong>

In [ ]:
embedding_features_test = []
for sent in test_statements:
    sent = [x.lower() for x in sent.split(' ') if x not in stopwords.words('english')]
    for i in range(len(sent)):
        sent[i] = model[sent[i]] if sent[i] in model.vocab else np.zeros(300)
    embedding_features_test.append(np.mean(sent, axis=0))
    
embedding_features_test = np.array(embedding_features_test)    

In [ ]:
X_train_emb = embedding_features_train
y_train_emb = train_labels

X_test_emb = embedding_features_test
y_test_emb = test_labels

<div class="task_description">
   <i class="subtask">11.6.3</i> <i class="l3">L3</i> <br>
</div>

Train the naive bayes classifier and evaluate it using confusion metric, accuracy and classification report. 

<strong style="color: blue">Lösung:</strong>

In [ ]:
clf_emb = GaussianNB()
clf_emb.fit(X_train_emb, y_train_emb)

predictions_emb = clf_emb.predict(X_test_emb)

In [ ]:
cf_emb = confusion_matrix(y_test_emb,predictions_emb)
print(cf_emb)

In [ ]:
sns.heatmap(cf_emb, cmap="GnBu", annot=True, fmt='g');

In [ ]:
accuracy_score(y_test_emb, predictions_emb) 

In [ ]:
print(classification_report(y_test_emb,predictions_emb))

<div class="task_description">
   <i class="subtask">11.6.4</i> <i class="l2">L2</i> <br>
</div>

Find three statements that the classifier fails.<br> Please run the following code snippet to test the classifier. Give a justification for the classifier failure. 

In [ ]:
def get_embedding_vector(input_text):
    dim = 300
    return np.mean([model[w] for w in input_text.split(' ') if w in model.vocab]
                            or [np.zeros(dim)], axis=0)


# Process user input. Processing continues until the user says goodbye. 
text = ""
while text != "goodbye":
    # Read user input
    text = input()
    test_features_emb = get_embedding_vector(text) 
    print('Positive Sentiment' if clf_emb.predict([test_features_emb]) == 1 else 'Negative sentiment')


<strong style="color: blue">Lösung:</strong>

i am not sad –– negation

product quality is okay okay –– interjection words

i hardly like the product –– hardly
    

<div class="task_description">
   <i class="subtask">11.6.5</i> <i class="l2">L2</i> <br>
</div>

The error analysis shows that despite a better performance model it is not working well with sentences having a negation. Understand and explain the idea behind the following function. 

In [ ]:
def toggle_prediction(prediction, text):
    prediction_toggled = [x for x in prediction]
    for i, sent in enumerate(text):
        sent = sent.lower().split(' ')
        if 'not' in sent:
            prediction_toggled[i] = 1 - prediction_toggled[i]
        else:
            for w in sent:
                if "n't" in w:
                    prediction_toggled[i] = 1 - prediction_toggled[i]
                    break
    return prediction_toggled
                
predictions_emb_toggled = toggle_prediction(predictions_emb, test_statements)

<strong style="color: blue">Lösung:</strong>

The function toggles the prediction if the test statement contains negations.
For example words like "not", "haven't", "aren't" etc.

<div class="task_description">
   <i class="subtask">11.6.6</i> <i class="l2">L2</i> <br>
</div>

Compare the performance with the original classfier prediction results and describe your observation.

<strong style="color: blue">Lösung:</strong>

In [ ]:
cf_emb = confusion_matrix(y_test_emb,predictions_emb_toggled)
print(cf_emb)

In [ ]:
sns.heatmap(cf_emb, cmap="GnBu", annot=True, fmt='g')

In [ ]:
accuracy_score(y_test_emb,predictions_emb_toggled) 

In [ ]:
print(classification_report(y_test_emb, predictions_emb_toggled))

<div class="task_description">
   <i class="subtask">11.6.7</i> <i class="l2">L2</i> <br>
</div>

Find three statements that the negation conditioned classifier fails. Please run the following code snippet to test the classifier. Give justifications for the classifier failure. 

In [ ]:
# Process user input. Processing continues until the user says goodbye. 
text = ""
while text != "goodbye":
    # Read user input
    text = input()
    test_features_emb = get_embedding_vector(text) 
    print('Positive Sentiment' if toggle_prediction(clf_emb.predict([test_features_emb]), [text])[0] == 1 else 'Negative sentiment')

<strong style="color: blue">Lösung:</strong>

1. Does Not Work.
2. My car will not accept this cassette.
3. It's not what it says it is.

### Error Analysis

<div class="task_description">
    <i class="task">Task 11.7:</i> <br>
</div>

<div class="task_description">
   <i class="subtask">11.7.1</i> <i class="l2">L2</i> <br>
</div>

Please go through the following code snippets. Which of them shows where toggling of prediction works and where not. Can you provide a comment based on the code results?

In [ ]:
no_example = 10

for i, sample in enumerate(X_test_emb):
    if "n't" in test_statements.values[i].lower() or "not" in test_statements.values[i].lower():
        if y_test_emb.values[i] != clf_emb.predict([sample])[0]:
            print("Text: %s" %test_statements.values[i])
            print("Ground Truth: %s" %y_test_emb.values[i])
            print("Prediction: %s" %clf_emb.predict([sample])[0])
            print("Toggled Prediction: %s" %int(toggle_prediction(clf_emb.predict([sample]), [test_statements.values[i]])[0]))
            print()
            no_example -= 1
            if no_example == 0:
                break

In [ ]:
no_example = 10

for i, sample in enumerate(X_test_emb):
    if "n't" in test_statements.values[i].lower() or "not" in test_statements.values[i].lower():
        if y_test_emb.values[i] != toggle_prediction(clf_emb.predict([sample]), [test_statements.values[i]]):
            print("Text: %s" %test_statements.values[i])
            print("Ground Truth: %s" %y_test_emb.values[i])
            print("Prediction: %s" %clf_emb.predict([sample])[0])
            print("Toggled Prediction: %s" %int(toggle_prediction(clf_emb.predict([sample]), [test_statements.values[i]])[0]))
            print()
            no_example -= 1
            if no_example == 0:
                break

<strong style="color: blue">Lösung:</strong>

In [ ]:
# toggling of prediction works

no_example = 10


for i, sample in enumerate(X_test_emb):
    if "n't" in test_statements.values[i].lower() or "not" in test_statements.values[i].lower():
        if y_test_emb.values[i] != clf_emb.predict([sample])[0]:
            print("Text: %s" %test_statements.values[i])
            print("Ground Truth: %s" %y_test_emb.values[i])
            print("Prediction: %s" %clf_emb.predict([sample])[0])
            print("Toggled Prediction: %s" %int(toggle_prediction(clf_emb.predict([sample]), [test_statements.values[i]])[0]))
            print()
            no_example -= 1
            if no_example == 0:
                break

In [ ]:
# toggling of prediction fails

no_example = 10


for i, sample in enumerate(X_test_emb):
    if "n't" in test_statements.values[i].lower() or "not" in test_statements.values[i].lower():
        if y_test_emb.values[i] != int(toggle_prediction(clf_emb.predict([sample]), [test_statements.values[i]])[0]):
            print("Text: %s" %test_statements.values[i])
            print("Ground Truth: %s" %y_test_emb.values[i])
            print("Prediction: %s" %clf_emb.predict([sample])[0])
            print("Toggled Prediction: %s" %int(toggle_prediction(clf_emb.predict([sample]), [test_statements.values[i]])[0]))
            print()
            no_example -= 1
            if no_example == 0:
                break

# Homework
<br>

<div class="task_description">
    <i class="task">Task 11.1:</i> <br>
</div>


You have seen how to solve the task by using the Word2Vec embeddings. Now try it yourself using the GloVe embeddings and compare the results. Elaborate your comparison briefly. Comparing only the accuracy is not enough.

We recommend using the glove.6B file you know from other exercises.
<a href="https://nlp.stanford.edu/projects/glove/">GloVe Website</a>


### *Submission guidelines*
* *The submission has to be done by a team of two people. **Individual submissions will not be graded**.*
* *Please state the **name and matriculation number of all team members** in every submission **clearly**.*
* *Only **one team member should submit** the homework. If more than one version of the same homework is submitted by accident (submitted by more than one group member), please reach out to a tutor **as soon as possible**. Otherwise, the first submitted homework will be graded.*
* *The submission must be in a Jupyter Notebook format (.ipynb). Submissions in **other formats will not be graded**.*
* *It is not necessary to also submit the part of the exercise discussed by the tutor, please only submit the homework part.*
* *If pictures need to be submitted, it is allowed to hand them in in a zip folder, together with the notebook. They should be added to the notebook like this: ``![example1](examplepicture1.PNG)`` (without apostrophs in a Markdown-Cell).*